In [32]:
from numba import jit,autojit,double

In [45]:
@jit(nopython=True, locals=dict(A=double[:]))
def numbasum(A):
    total = 0
    for i in xrange(len(A)):
        total += A[i]
    return total

In [3]:
def pysum(A):
    total = 0
    for i in xrange(len(A)):
        total += A[i]
    return total

In [4]:
import numpy as np

In [6]:
A = np.random.random_integers(-9,9,100000)

In [48]:
%%time 
A = np.random.random_integers(-9,9,20000000).astype(np.float64)
numbasum(A)

CPU times: user 436 ms, sys: 36 ms, total: 472 ms
Wall time: 472 ms


In [40]:
%%time 
A = np.random.random_integers(-9,9,20000000)
pysum(A)

CPU times: user 3.17 s, sys: 28 ms, total: 3.2 s
Wall time: 3.14 s


In [30]:
np.float64

numpy.float64

In [50]:
import numpy as np

In [49]:
numbasum.inspect_types()

numbasum (array(float64, 1d, C),)
--------------------------------------------------------------------------------
# File: <ipython-input-45-118662c6f54c>
# --- LINE 1 --- 

@jit(nopython=True, locals=dict(A=double[:]))

# --- LINE 2 --- 

def numbasum(A):

    # --- LINE 3 --- 
    # label 0
    #   A = arg(0, name=A)  :: array(float64, 1d, A)
    #   $const0.1 = const(int, 0)  :: int64
    #   total = $const0.1  :: float64
    #   del $const0.1

    total = 0

    # --- LINE 4 --- 
    #   jump 6
    # label 6
    #   $6.1 = global(xrange: <type 'xrange'>)  :: range
    #   $6.2 = global(len: <built-in function len>)  :: len
    #   $6.4 = call $6.2(A)  :: (array(float64, 1d, A),) -> int64
    #   del $6.2
    #   $6.5 = call $6.1($6.4)  :: (int64,) -> range_state_int64
    #   del $6.4
    #   del $6.1
    #   $6.6 = getiter(value=$6.5)  :: range_iter_int64
    #   del $6.5
    #   $phi25.1 = $6.6  :: range_iter_int64
    #   del $6.6
    #   jump 25
    # label 25
    #   $25.2 = i

In [57]:
def foo(n):
    my_list = []
    for i in xrange(n):
        num = np.random.randint()
        my_list.append(num)
    evens = 0
    for j in xrange(n):
        if j%2 == 0:
            total += my_list[j]
    return mylist, evens
    

In [59]:
%time foo(1e6)

CPU times: user 420 ms, sys: 0 ns, total: 420 ms
Wall time: 419 ms


-249733

In [75]:
%prun foo(int(1e6))

In [74]:
def foo_fast(n):
    my_list = np.random.random_integers(-9,9,n)
    return my_list, np.sum(my_list[::2])

In [76]:
%time foo_fast(int(1e6))

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 28.4 ms


(array([ 1,  9, -1, ..., -1, -6, -1]), -150)

In [77]:
%prun foo_fast(int(1e6))

In [78]:
def LU_part1(A):
    """Returns the LU decomposition of a square matrix."""
    n = A.shape[0]
    U = np.array(np.copy(A), dtype=float)
    L = np.eye(n)
    
    for i in xrange(1,n):
        for j in xrange(i):
            L[i,j] = U[i,j]/U[j,j]
            for k in xrange(j,n):
                U[i,k] -= L[i,j]*U[j,k]
    return L,U

In [79]:
def LU(A):
    """Returns the LU decomposition of a square matrix."""
    n = A.shape[0]
    U = np.array(np.copy(A), dtype=float)
    L = np.eye(n)
    
    for i in xrange(1,n):
        for j in xrange(i):
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:] -= L[i,j]*U[j,j:]
    return L,U

In [80]:
A = np.random.random_integers(-9,9,25).reshape((5,5))

In [81]:
np.allclose(LU_part1(A),LU(A))

True

In [82]:
load_ext Cython

In [85]:
%%cython
import numpy as np
def cy_LU(double[:,:] A):
    cdef int n = A.shape[0]
    cdef double[:,:] U = np.array(np.copy(A))
    cdef double[:,:] L = np.eye(n).astype(float)
    cdef int i,j,k
    
    for i in xrange(1,n):
        for j in xrange(i):
            L[i,j] = U[i,j]/U[j,j]
            for k in xrange(j,n):
                U[i,k] -= L[i,j]*U[j,k]
    return L,U

In [86]:
def LU_part1(A):
    """Returns the LU decomposition of a square matrix."""
    n = A.shape[0]
    U = np.array(np.copy(A), dtype=float)
    L = np.eye(n)
    
    for i in xrange(1,n):
        for j in xrange(i):
            L[i,j] = U[i,j]/U[j,j]
            for k in xrange(j,n):
                U[i,k] -= L[i,j]*U[j,k]
    return L,U

In [92]:
n = 200
A = np.random.random_integers(-9,9,n**2).astype(float).reshape((n,n))

In [88]:
%time L,U = LU_part1(A)

CPU times: user 1.34 s, sys: 12 ms, total: 1.36 s
Wall time: 1.34 s


In [93]:
%time L,U = cy_LU(A)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.9 ms


In [94]:
np.allclose(cy_LU(A),LU_part1(A))

True

In [96]:
%%cython
import numpy as np
def cytridiag(double[:] a, double[:] b, double[:] c, double[:] d):
    """Solve the tridiagonal system Ax = d where A has diagonals a, b, and c.

    Inputs:
        a, b, c, d (array) - All 1-D NumPy arrays of equal length.
    Returns:
        x (array) - solution to the tridiagonal system.
    """
    cdef int n = a.size

    # Make copies so the original arrays remain unchanged
    cdef double[:] aa = np.copy(a)
    cdef double[:] bb = np.copy(b)
    cdef double[:] cc = np.copy(c)
    cdef double[:] dd = np.copy(d)

    cdef int i
    cdef double temp
    # Forward sweep
    for i in xrange(1, n):
        temp = aa[i]/bb[i-1]
        bb[i] = bb[i] - temp*cc[i-1] 
        dd[i] = dd[i] - temp*dd[i-1]

    cdef double[:] x
    # Back substitution
    x = np.zeros_like(a)
    x[-1] = dd[-1]/bb[-1]
    for i in xrange(n-2, -1, -1):
        x[i] = (dd[i]-cc[i]*x[i+1])/bb[i]

    return x

In [101]:
def init_tridiag(n):
    """Initializes a random nxn tridiagonal matrix A.
    
    Inputs:
        n (int) : size of array
    
    Returns:
        a (1-D array) : (-1)-th diagonal of A
        b (1-D array) : main diagonal of A
        c (1-D array) : (1)-th diagonal of A
        A (2-D array) : nxn tridiagonal matrix defined by a,b,c. 
    """
    a = np.random.random_integers(1,9,n).astype("float")
    b = np.random.random_integers(1,9,n).astype("float")
    c = np.random.random_integers(1,9,n).astype("float")

    A = np.zeros((b.size,b.size))
    np.fill_diagonal(A,b)
    np.fill_diagonal(A[1:,:-1],a[1:])
    np.fill_diagonal(A[:-1,1:],c)
    return a,b,c,A

In [105]:
n = 1000
a,b,c,A = init_tridiag(n)
d = np.random.random_integers(-9,9,n).astype(float)

In [106]:
def pytridiag(a,b,c,d):
    """Solve the tridiagonal system Ax = d where A has diagonals a, b, and c.
    
    Inputs:
        a, b, c, d (array) - All 1-D NumPy arrays of equal length.
    Returns:
        x (array) - solution to the tridiagonal system.
    """
    n = len(a)
    
    # Make copies so the original arrays remain unchanged
    aa = np.copy(a)
    bb = np.copy(b)
    cc = np.copy(c)
    dd = np.copy(d)
    
    # Forward sweep
    for i in xrange(1, n):
        temp = aa[i]/bb[i-1]
        bb[i] = bb[i] - temp*cc[i-1] 
        dd[i] = dd[i] - temp*dd[i-1]

    # Back substitution
    x = np.zeros_like(a)
    x[-1] = dd[-1]/bb[-1]
    for i in xrange(n-2, -1, -1):
        x[i] = (dd[i]-cc[i]*x[i+1])/bb[i]

    return x

In [107]:
%time pytridiag(a,b,c,d)

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 4.51 ms


array([ -1.10420777e-01,  -9.36902413e-01,   1.51871084e+00,
        -3.30893480e-01,   3.29347180e+00,   7.02868416e-01,
        -2.82098909e+00,  -5.61279187e-01,   1.43748585e+00,
         2.01318691e+00,  -1.75292414e+00,   3.87623343e-01,
         2.97023660e+00,  -3.34715331e+00,   1.25383343e+00,
        -1.19877428e+00,   3.28130168e+00,  -1.75225972e+00,
        -2.31647427e+00,   2.19351593e+00,   1.86887504e+00,
        -4.27408285e+00,   6.52473492e+00,  -1.12758217e+01,
         7.16098074e+00,   6.89567052e-01,  -8.86900371e+00,
         1.42883799e+01,  -1.14193762e+01,  -2.76632678e+00,
         1.97986204e+01,  -3.85014985e+00,   8.17376299e-01,
         1.94293849e+00,  -5.27062957e+00,   1.71244368e+00,
         9.93926899e+00,  -1.04142314e+01,  -9.12989869e+00,
         4.39420047e+01,  -1.32065253e+01,  -3.29078401e+01,
         7.64705742e+01,  -9.12391519e+00,  -2.47925268e+00,
         1.66375572e+01,  -4.13764582e-01,  -3.65227932e+00,
         7.58132803e+00,

In [110]:
np.allclose(pytridiag(a,b,c,d),np.array(cytridiag(a,b,c,d)))

True